In [58]:
!pip install torch torchvision pillow


In [59]:
!pip install numpy
import heapq
import numpy as np
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from scipy.spatial.distance import cosine


In [60]:

def load_model():
    model = resnet50(pretrained=True)
    model.eval()
    return model


In [61]:
!pip install scikit-learn


In [62]:
# Initialize model and transformations

model=load_model()

In [63]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [64]:
import numpy as np
def get_image_feature_vector(img_path, model, transform):
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img)
    #print(img_t)
    batch_t = torch.unsqueeze(img_t, 0)
    with torch.no_grad():
        out = model(batch_t)
    return out[0]


In [65]:
def precision_at_k(retrieved, relevant, k):
    if k > len(retrieved):
        k = len(retrieved)
    retrieved_at_k = retrieved[:k]
    
    hits = sum([1 for r in retrieved_at_k if r in relevant])
    return hits / k if k > 0 else 0

def recall_at_k(retrieved, relevant, k):
    if k > len(retrieved):
        k = len(retrieved)
    retrieved_at_k = retrieved[:k]
    hits = 0
    for r in retrieved_at_k:
        if r in relevant:
            hits += 1
    #hits = sum([1 for r in retrieved_at_k if r in relevant])
    return hits / len(relevant) if len(relevant) > 0 else 0

In [66]:
ground_truth = {
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/bear.jpg': {
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4870.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4871.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4873.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4875.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4872.jpg'
    },
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/cat.jpg': {
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4857.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4858.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4856.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4860.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4861.jpg'
        
    }
    ,
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/dog.jpg': {
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4855.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4838.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4839.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4849.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/4851.jpg'
        
    },
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/tiger.jpg': {
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/48.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/47.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/27.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/247.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/0.jpg'
        
    },
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/lion.jpg': {
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/888.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/898.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/899.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/901.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/907.jpg',
        'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy/1052.jpg'
        
        
    },
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/human.jpg':{
        
    }
}


In [77]:
# Query Image path:

test_data = ['C:/Users/ajayk/Downloads/unt/irs/archive/query/dog.jpg'
             ]


In [78]:
import numpy as np

def find_most_similar_images(input_img_path, folder_path, model, transform, top_k=3):
    input_feature = get_image_feature_vector(input_img_path, model, transform)
    top_images = []

    for img_name in os.listdir(folder_path):
        #img_path = os.path.join(folder_path, img_name)
        img_path = os.path.join(folder_path, img_name).replace('\\', '/')
        if img_path == input_img_path:
            continue

        feature = get_image_feature_vector(img_path, model, transform)
        dist = cosine(input_feature, feature)
        if len(top_images) < top_k:
            heapq.heappush(top_images, (-dist, img_path))  # Use negative distance for max heap
        else:
            heapq.heappushpop(top_images, (-dist, img_path))  # Replace if new distance is smaller

    # Return paths of top k images
    return [img for _, img in sorted(top_images, reverse=True)]


def test_cbir_performance(model, transform, test_data, ground_truth, k=3):
    precisions = []
    recalls = []

    for test_img in test_data:
        
        most_similar_images = find_most_similar_images(test_img, folder_path, model, transform, top_k=k)
        for similar_image in most_similar_images:
            output_path = os.path.join(output_folder, os.path.basename(similar_image))
            Image.open(similar_image).save(output_path)


        relevant_images = ground_truth[test_img]
        #print(most_similar_images, relevant_images)
        precisions.append(precision_at_k(most_similar_images, relevant_images, k))
        recalls.append(recall_at_k(most_similar_images, relevant_images, k))

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    return avg_precision, avg_recall


# Input image and folder paths
folder_path = 'C:/Users/ajayk/Downloads/unt/irs/archive/data'
output_folder = 'C:/Users/ajayk/Downloads/unt/irs/archive/output'


avg_precision, avg_recall = test_cbir_performance(model, transform, test_data, ground_truth)


In [79]:




def find_most_similar_images(input_img_path, folder_path, model, transform, top_k=3):
    input_feature = get_image_feature_vector(input_img_path, model, transform)
    top_images = []

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name).replace('\\', '/')
        if img_path == input_img_path:
            continue

        feature = get_image_feature_vector(img_path, model, transform)
        dist = cosine(input_feature, feature)
        if len(top_images) < top_k:
            heapq.heappush(top_images, (-dist, img_path))  # Use negative distance for max heap
        else:
            heapq.heappushpop(top_images, (-dist, img_path))  # Replace if new distance is smaller

    # Return paths of top k images
    return [img for _, img in sorted(top_images, reverse=True)]



def test_cbir_performance(model, transform, test_data, ground_truth, k=3):
    precisions = []
    recalls = []

    for test_img in test_data:
        most_similar_images = find_most_similar_images(test_img, folder_path, model, transform, top_k=k)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print(f"Created output folder: {output_folder}")

        for similar_image in most_similar_images:
            output_path = os.path.join(output_folder, os.path.basename(similar_image))
            try:
                Image.open(similar_image).save(output_path)
                #print(f"Similar image saved to {output_path}")
            except Exception as e:
                print(f"Failed to save image {similar_image}. Error: {e}")
        for similar_image in most_similar_images:
            output_path = os.path.join(output_folder, os.path.basename(similar_image))
            Image.open(similar_image).save(output_path)

        
        
        relevant_images = ground_truth[test_img]

        precisions.append(precision_at_k(most_similar_images, relevant_images, k))
        recalls.append(recall_at_k(most_similar_images, relevant_images, k))

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    return avg_precision, avg_recall



# Example usage
test_data = [
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/bear.jpg',
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/cat.jpg',
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/dog.jpg',
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/tiger.jpg',
    'C:/Users/ajayk/Downloads/unt/irs/archive/query/lion.jpg'
]

folder_path = 'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy'
output_folder = 'C:/Users/ajayk/Downloads/unt/irs/archive/accuracy'

avg_precision, avg_recall = test_cbir_performance(model, transform, test_data, ground_truth)
print(f"Average Precision@3: {avg_precision}")
print(f"Average Recall@3: {avg_recall}")


Average Precision@3: 0.7333333333333333
Average Recall@3: 0.44000000000000006


In [75]:
def cbir_accuracy(test_data, ground_truth, folder_path, model, transform, k=3):
    correct_retrievals = 0

    for test_img in test_data:
        most_similar_images = find_most_similar_images(test_img, folder_path, model, transform, top_k=k)
        relevant_images = ground_truth[test_img]

        # Check if at least one retrieved image is relevant
        if any(img in relevant_images for img in most_similar_images):
            correct_retrievals += 1

    return correct_retrievals / len(test_data) if len(test_data) > 0 else 0

# Usage
accuracy = cbir_accuracy(test_data, ground_truth, folder_path, model, transform, k=3)
print(f"CBIR Accuracy: {accuracy}")


CBIR Accuracy: 0.8


In [76]:
def f1_score_at_k(retrieved, relevant, k):
    precision = precision_at_k(retrieved, relevant, k)
    recall = recall_at_k(retrieved, relevant, k)

    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def calculate_f1_score(model, transform, test_data, ground_truth, k=3):
    f1_scores = []

    for test_img in test_data:
        most_similar_images = find_most_similar_images(test_img, folder_path, model, transform, top_k=k)
        relevant_images = ground_truth[test_img]

        f1 = f1_score_at_k(most_similar_images, relevant_images, k)
        f1_scores.append(f1)

    avg_f1_score = np.mean(f1_scores) if f1_scores else 0
    return avg_f1_score

# Usage
avg_f1_score = calculate_f1_score(model, transform, test_data, ground_truth, k=3)
print(f"Average F1 Score@3: {avg_f1_score}")


Average F1 Score@3: 0.55
